In [1]:
import numpy as np
import scipy as sp
import pandas as pd
#import pandas_ml as pdml
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV



In [2]:
gameteamDF = pd.read_csv("gamegoaliestats.csv")
gameDF = pd.read_csv("game.csv")

In [3]:
#sorting merged data table by teamID, then gameID
gameteamDF.sort_values(by=["team_id", "game_id"], inplace=True)

In [4]:
#first need to separate composite data table into teams
teamNums = gameteamDF["team_id"].unique()
print(teamNums)
#creating array to hold team DFs
teamDF = [None]*55

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 52 53 54]


In [5]:
for team in teamNums:
    #putting DF in index that matches teamID
    buff = gameteamDF["team_id"] == team
    thisTeamDF = gameteamDF[buff]
    
    #adding needed columns
    thisTeamDF = thisTeamDF.reindex(columns = thisTeamDF.columns.tolist() +
                                   ["rAvgGoals", "rAvgShots", "rAvgHits", "rAvgPIM", "rAvgPPchance",
                                   "rAvgPPgoals", "rAvgFO%", "rAvgGive", "rAvgTake", "rAvgShotsVs",
                                   "rAvgSaves", "rAvgSave%"])
    
    #getting running average of game stats up to and including current game
    thisTeamDF["rAvgGoals"] = thisTeamDF["goals_x"].ewm(span=40).mean()
    thisTeamDF["rAvgShots"] = thisTeamDF["shots_x"].ewm(span=40).mean()
    thisTeamDF["rAvgHits"] = thisTeamDF["hits"].ewm(span=40).mean()
    thisTeamDF["rAvgPIM"] = thisTeamDF["pim_x"].ewm(span=40).mean()
    thisTeamDF["rAvgPPchance"] = thisTeamDF["powerPlayOpportunities"].ewm(span=40).mean()
    thisTeamDF["rAvgPPgoals"] = thisTeamDF["powerPlayGoals"].ewm(span=40).mean()
    thisTeamDF["rAvgFO%"] = thisTeamDF["faceOffWinPercentage"].ewm(span=40).mean()
    thisTeamDF["rAvgGive"] = thisTeamDF["giveaways"].ewm(span=40).mean()
    thisTeamDF["rAvgTake"] = thisTeamDF["takeaways"].ewm(span=40).mean()
    thisTeamDF["rAvgShotsVs"] = thisTeamDF["shots_y"].ewm(span=40).mean()
    thisTeamDF["rAvgSaves"] = thisTeamDF["saves"].ewm(span=40).mean()
    thisTeamDF["rAvgSave%"] = thisTeamDF["savePercentage"].ewm(span=40).mean()
    
    #removing columns we're not using
    thisTeamDF.drop(["timeOnIce", "powerPlaySaves", "shortHandedSaves",
                    "evenSaves", "shortHandedShotsAgainst", "evenShotsAgainst",
                    "powerPlayShotsAgainst", "savePercentage", "powerPlaySavePercentage",
                    "evenStrengthSavePercentage"],axis=1)
    
    teamDF[team] = thisTeamDF
#print(numGames.head())

In [6]:
#recombining team data tables into composite again
combineDF = teamDF[1]
for i in range(2,len(teamDF)):
    combineDF = combineDF.append(teamDF[i])

In [7]:
#extracting only required columns from game table
gameDF2 = gameDF.filter(["game_id", "away_team_id", "home_team_id", "outcome"], axis=1)

In [8]:
#merging game table (which indexes the games) with the game stats table
testDF = gameDF2.merge(combineDF, left_on=['game_id','home_team_id'],right_on=['game_id','team_id']).merge(combineDF, left_on=['game_id','away_team_id'],right_on=['game_id','team_id'])

In [9]:
pd.set_option('display.max_columns', None)

In [10]:
#function to convert 6 class target into 2 classes
def shortenOutcome(x):
    return x[:9]


In [11]:
#apply function
testDF["outcome"] = testDF["outcome"].apply(shortenOutcome)

In [12]:
testDF

,game_id,away_team_id,home_team_id,outcome,team_id_x,goals_x_x,shots_x_x,hits_x,pim_x_x,powerPlayOpportunities_x,powerPlayGoals_x,faceOffWinPercentage_x,giveaways_x,takeaways_x,timeOnIce_x,shots_y_x,saves_x,powerPlaySaves_x,shortHandedSaves_x,evenSaves_x,shortHandedShotsAgainst_x,evenShotsAgainst_x,powerPlayShotsAgainst_x,savePercentage_x,powerPlaySavePercentage_x,evenStrengthSavePercentage_x,rAvgGoals_x,rAvgShots_x,rAvgHits_x,rAvgPIM_x,rAvgPPchance_x,rAvgPPgoals_x,rAvgFO%_x,rAvgGive_x,rAvgTake_x,rAvgShotsVs_x,rAvgSaves_x,rAvgSave%_x,team_id_y,goals_x_y,shots_x_y,hits_y,pim_x_y,powerPlayOpportunities_y,powerPlayGoals_y,faceOffWinPercentage_y,giveaways_y,takeaways_y,timeOnIce_y,shots_y_y,saves_y,powerPlaySaves_y,shortHandedSaves_y,evenSaves_y,shortHandedShotsAgainst_y,evenShotsAgainst_y,powerPlayShotsAgainst_y,savePercentage_y,powerPlaySavePercentage_y,evenStrengthSavePercentage_y,rAvgGoals_y,rAvgShots_y,rAvgHits_y,rAvgPIM_y,rAvgPPchance_y,rAvgPPgoals_y,rAvgFO%_y,rAvgGive_y,rAvgTake_y,rAvgShotsVs_y,rAvgSaves_y,rAvgSave%_y
0,2011030221,1,4,home win,4,4.0,36.0,27.0,6.0,6.0,1.0,55.1,13.0,4.0,3876.0,26.0,23.0,4.0,5.0,14.0,5.0,16.0,5.0,88.461538,80.000000,87.500000,3.528457,30.822251,29.300803,18.635282,3.901151,1.049185,48.829660,7.873883,5.358081,26.962167,24.281280,89.192478,1,3.0,26.0,31.0,12.0,3.0,1.0,44.9,6.0,7.0,3876.0,36.0,32.0,7.0,0.0,25.0,0.0,28.0,8.0,88.888889,87.500000,89.285714,2.683530,28.846294,25.473390,10.614662,3.202568,0.662760,47.308940,7.238263,6.484754,26.658643,24.617180,91.541293
1,2011030222,1,4,away win,4,1.0,20.0,24.0,32.0,5.0,0.0,49.1,9.0,6.0,3559.0,34.0,31.0,6.0,0.0,25.0,0.0,28.0,6.0,91.176471,100.000000,89.285714,3.405105,30.294280,29.042200,19.287289,3.954759,0.997999,48.842848,7.928821,5.389397,27.305512,24.609057,89.289269,1,4.0,35.0,32.0,12.0,4.0,0.0,50.9,8.0,7.0,3599.0,20.0,19.0,1.0,3.0,15.0,3.0,16.0,1.0,95.000000,100.000000,93.750000,2.747759,29.146527,25.791817,10.682251,3.241474,0.630424,47.484144,7.275427,6.509892,26.333774,24.343123,91.710040
2,2011030223,4,1,home win,1,4.0,31.0,30.0,10.0,2.0,1.0,49.2,11.0,4.0,4641.0,28.0,25.0,4.0,0.0,21.0,0.0,23.0,5.0,89.285714,80.000000,91.304348,2.808854,29.236955,25.997129,10.648965,3.180904,0.648455,47.567858,7.457144,6.387438,26.415067,24.375171,91.591760,4,3.0,28.0,28.0,4.0,5.0,1.0,50.8,2.0,1.0,4623.0,31.0,27.0,1.0,1.0,25.0,1.0,28.0,2.0,87.096774,50.000000,89.285714,3.385342,30.182353,28.991356,18.541492,4.005752,0.998097,48.938329,7.639581,5.175259,27.485749,24.725700,89.182307
3,2011030224,4,1,home win,1,4.0,43.0,19.0,4.0,5.0,1.0,37.5,5.0,3.0,3600.0,22.0,20.0,1.0,1.0,18.0,2.0,18.0,2.0,90.909091,50.000000,100.000000,2.866968,29.908430,25.655751,10.324574,3.269654,0.665607,47.076666,7.337264,6.222171,26.199664,24.161714,91.558454,4,2.0,22.0,23.0,10.0,2.0,1.0,62.5,20.0,5.0,3555.0,42.0,39.0,6.0,0.0,33.0,0.0,35.0,7.0,92.857143,85.714286,94.285714,3.317758,29.783175,28.699067,18.124794,3.907901,0.998190,49.599937,8.242586,5.166709,28.193829,25.422074,89.361584
4,2011030225,1,4,away win,4,1.0,28.0,38.0,8.0,1.0,0.0,45.0,14.0,7.0,3452.0,30.0,27.0,2.0,1.0,24.0,1.0,26.0,3.0,90.000000,66.666667,92.307692,3.204686,29.696183,29.152812,17.630856,3.766039,0.949493,49.375530,8.523461,5.256146,28.281943,25.499053,89.392730,1,3.0,30.0,26.0,2.0,4.0,1.0,55.0,6.0,5.0,3600.0,28.0,27.0,0.0,3.0,24.0,3.0,25.0,0.0,96.428571,NaN,96.000000,2.873458,29.912897,25.672546,9.918436,3.305286,0.681921,47.463228,7.272022,6.162544,26.287498,24.300188,91.796057
5,2011030411,26,1,away win,1,1.0,17.0,44.0,2.0,2.0,0.0,44.6,11.0,8.0,4093.0,25.0,23.0,1.0,0.0,22.0,0.0,24.0,1.0,92.000000,100.000000,91.666667,2.715301,28.484575,25.173347,9.436285,3.364166,0.606872,47.930836,7.405991,5.933139,26.584134,24.590923,91.994232,26,2.0,25.0,35.0,4.0,1.0,0.0,55.4,6.0,4.0,4093.0,17.0,16.0,2.0,1.0,13.0,1.0,14.0,2.0,94.117647,100.000000,92.857143,2.719107,32.077578,32.481123,12.090617,4.151726,0.517114,50.718525,8.651501,5.309090,26.177160,24.448400,93.098912
6,2011030412,26,1,away win,1,1.0,33.0,35.0,4.0,4.0

In [13]:
#extracting all required columns
df = testDF.filter(["outcome", "rAvgGoals_x", "rAvgShots_x", "rAvgHits_x", "rAvgPIM_x", "rAvgPPchance_x", "rAvgPPgoals_x",
                    "rAvgFO%_x", "rAvgGive_x", "rAvgTake_x", "rAvgShotsVs_x", "rAvgSaves_x", "rAvgSave%_x",
                   "rAvgGoals_y", "rAvgShots_y", "rAvgHits_y", "rAvgPIM_y", "rAvgPPchance_y", "rAvgPPgoals_y",
                    "rAvgFO%_y", "rAvgGive_y", "rAvgTake_y", "rAvgShotsVs_y", "rAvgSaves_y", "rAvgSave%_y"])

In [14]:
#separating target column
df2 = df.drop(['outcome'],axis = 1)

In [15]:
num_features = ["rAvgGoals_x", "rAvgShots_x", "rAvgHits_x", "rAvgPIM_x", "rAvgPPchance_x", "rAvgPPgoals_x",
                    "rAvgFO%_x", "rAvgGive_x", "rAvgTake_x", "rAvgShotsVs_x", "rAvgSaves_x", "rAvgSave%_x",
                   "rAvgGoals_y", "rAvgShots_y", "rAvgHits_y", "rAvgPIM_y", "rAvgPPchance_y", "rAvgPPgoals_y",
                    "rAvgFO%_y", "rAvgGive_y", "rAvgTake_y", "rAvgShotsVs_y", "rAvgSaves_y", "rAvgSave%_y"]

In [16]:
#normalizing values in features to [0-1]
scaled_features = {}

for each in num_features:
    maxNum, minNum = df2[each].max(), df2[each].min()
    scaled_features[each] = [maxNum, minNum]
    df2.loc[:, each] = (df2[each] - minNum)/(maxNum-minNum)

In [17]:
outcomes = df['outcome']

In [18]:
#split dataset
x_train,x_test,y_train,y_test = train_test_split(df2, outcomes, test_size=.2, random_state=0);

In [19]:
# Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 28)
clf_B = SVC(random_state = 68, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)
clf_D = GaussianNB()
clf_E = MLPClassifier(random_state=99)
clf_F = RandomForestClassifier(random_state=88)

In [20]:
clf_A.fit(x_train, y_train)

print("Accuracy LR on training set: {:.3f}".format(clf_A.score(x_train, y_train)))
print("Accuracy LR on test set: {:.3f}".format(clf_A.score(x_test, y_test)))

clf_B.fit(x_train, y_train)

print("Accuracy SVC on training set: {:.3f}".format(clf_B.score(x_train, y_train)))
print("Accuracy SVC on test set: {:.3f}".format(clf_B.score(x_test, y_test)))

clf_C.fit(x_train, y_train)

print("Accuracy xgb on training set: {:.3f}".format(clf_C.score(x_train, y_train)))
print("Accuracy xgb on test set: {:.3f}".format(clf_C.score(x_test, y_test)))

clf_D.fit(x_train, y_train)

print("Accuracy NB on training set: {:.3f}".format(clf_D.score(x_train, y_train)))
print("Accuracy NB on test set: {:.3f}".format(clf_D.score(x_test, y_test)))

clf_E.fit(x_train, y_train)

print("Accuracy MLP on training set: {:.3f}".format(clf_E.score(x_train, y_train)))
print("Accuracy MLP on test set: {:.3f}".format(clf_E.score(x_test, y_test)))

clf_F.fit(x_train, y_train)

print("Accuracy MLP on training set: {:.3f}".format(clf_F.score(x_train, y_train)))
print("Accuracy MLP on test set: {:.3f}".format(clf_F.score(x_test, y_test)))

C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy LR on training set: 0.660
Accuracy LR on test set: 0.663


C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy SVC on training set: 0.648
Accuracy SVC on test set: 0.642
Accuracy xgb on training set: 0.702
Accuracy xgb on test set: 0.641
Accuracy NB on training set: 0.643
Accuracy NB on test set: 0.644
Accuracy MLP on training set: 0.662
Accuracy MLP on test set: 0.660


C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy MLP on training set: 0.985
Accuracy MLP on test set: 0.614


In [21]:
#grid search with cross validation on kernel paramters for SVC
svcParamGrid = { "kernel": ["linear", "rbf", "poly"]}

grid_search = GridSearchCV(SVC(), svcParamGrid, cv=5)
grid_search.fit(x_train, y_train)

print("best svc parameters: {}".format(grid_search.best_params_))
print("best cross valid score: {:.2f}".format(grid_search.best_score_))
print("test set score: {:.3f}".format(grid_search.score(x_test, y_test)))

C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\qvbingo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: Fut

best svc parameters: {'kernel': 'linear'}
best cross valid score: 0.66
test set score: 0.661


In [24]:
#grid search with cross validation on n_estimators and max_depth paramters for xgb
xgbParamGrid = { "n_estimators": [50, 100, 250, 500],
                "max_depth": [3, 4, 5, 6]}

grid_search = GridSearchCV(xgb.XGBClassifier(seed=77), xgbParamGrid, cv=5)
grid_search.fit(x_train, y_train)

print("best xgb parameters: {}".format(grid_search.best_params_))
print("best cross valid score: {:.2f}".format(grid_search.best_score_))
print("test set score: {:.3f}".format(grid_search.score(x_test, y_test)))

best xgb parameters: {'max_depth': 3, 'n_estimators': 50}
best cross valid score: 0.65
test set score: 0.643


In [25]:
SVCclass = SVC(random_state = 64, kernel='linear')
XGBclass = xgb.XGBClassifier(seed = 75, n_estimators=50, max_depth=3)

In [26]:
SVCclass.fit(x_train, y_train)
XGBclass.fit(x_train, y_train)

print("Accuracy SVC on training set: {:.3f}".format(SVCclass.score(x_train, y_train)))
print("Accuracy SVC on test set: {:.3f}".format(SVCclass.score(x_test, y_test)))

print("Accuracy xgb on training set: {:.3f}".format(XGBclass.score(x_train, y_train)))
print("Accuracy xgb on test set: {:.3f}".format(XGBclass.score(x_test, y_test)))

Accuracy SVC on training set: 0.660
Accuracy SVC on test set: 0.661
Accuracy xgb on training set: 0.684
Accuracy xgb on test set: 0.643
